In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fake-news/train.csv
/kaggle/input/fake-news/submit.csv
/kaggle/input/fake-news/test.csv


In [3]:
import pandas as pd
df=pd.read_csv('../input/fake-news/train.csv')

In [4]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
df=df.dropna()

In [6]:
x=df.drop('label',axis=1)

In [7]:
y=df['label']

In [8]:
y.value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [9]:
x.shape

(18285, 4)

In [10]:
y.shape

(18285,)

In [11]:
df.head(10)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi...",0
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi...",0
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...,0
10,10,Obama’s Organizing for Action Partners with So...,Aaron Klein,"Organizing for Action, the activist group that...",0
11,11,"BBC Comedy Sketch ""Real Housewives of ISIS"" Ca...",Chris Tomlinson,The BBC produced spoof on the “Real Housewives...,0


In [12]:
import tensorflow as tf

In [13]:
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [14]:
voc_size=10000

DATA PREPROCESSING

In [15]:
mes=x.copy()
mes['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [16]:
mes.reset_index(inplace=True)

In [17]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
from nltk.stem.porter import PorterStemmer

STOPWORDS AND STEMMING

In [19]:
ps=PorterStemmer()
corpus=[]

for i in range(0,len(mes)):
    review=re.sub('[^a-zA-Z]',' ',mes['title'][i])
    review=review.lower()
    review=review.split()
    review=[ps.stem(word) for word in review if word not in stopwords.words('english')]
    review=' '.join(review)
    corpus.append(review)

In [20]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

ONE HOT REPRESENTATION

In [21]:
one_hot_rep=[one_hot(word,voc_size) for word in corpus]
one_hot_rep

[[2430, 4417, 7574, 8084, 7960, 3228, 3219, 9583, 3284, 1365],
 [7310, 1906, 2324, 3419, 6298, 2643, 5253],
 [6402, 2711, 5826, 7624],
 [1550, 1440, 5041, 1281, 1336, 1936],
 [8185, 6298, 9558, 797, 9151, 4389, 6298, 5915, 9690, 8585],
 [3401,
  9280,
  9018,
  9829,
  2153,
  3697,
  8989,
  1721,
  8161,
  8252,
  7221,
  2670,
  9496,
  3396,
  5253],
 [4975, 8267, 5364, 797, 3692, 8408, 8403, 9665, 8633, 246, 8865],
 [4835, 7154, 4258, 1164, 5383, 4577, 3697, 2078, 8633, 246, 8865],
 [9713, 4346, 2852, 6267, 1149, 9864, 2561, 4228, 3697, 565],
 [6852, 1042, 9327, 6902, 6267, 4776, 8174, 3528],
 [7716, 8493, 4153, 649, 597, 4291, 4978, 9539, 4732, 8454, 4578],
 [1281, 8984, 7960, 9864, 3697, 5383],
 [239, 879, 473, 422, 2518, 4409, 1661, 9691, 4337],
 [6992, 9904, 2001, 4509, 2494, 5771, 1944, 8633, 246, 8865],
 [3241, 9496, 8106, 9455, 3708, 8633, 246, 8865],
 [397, 9398, 912, 4643, 2632, 6585, 4877, 7289, 8855, 2158],
 [1355, 2991, 1906],
 [4912, 9227, 1782, 5279, 3697, 489, 106, 

WORD EMBEDDING

In [22]:
sent_len=20
emb_doc=pad_sequences(one_hot_rep,padding='pre',maxlen=sent_len)
print(emb_doc)

[[   0    0    0 ... 9583 3284 1365]
 [   0    0    0 ... 6298 2643 5253]
 [   0    0    0 ... 2711 5826 7624]
 ...
 [   0    0    0 ... 8633  246 8865]
 [   0    0    0 ... 9756 4770 4922]
 [   0    0    0 ... 5544 4643 9865]]


In [23]:
emb_doc[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 2430,
       4417, 7574, 8084, 7960, 3228, 3219, 9583, 3284, 1365], dtype=int32)

In [24]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_len))
model.add(Bidirectional(LSTM(100)))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            400000    
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               112800    
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 201       
Total params: 513,001
Trainable params: 513,001
Non-trainable params: 0
_________________________________________________________________
None


In [25]:
len(emb_doc),y.shape

(18285, (18285,))

In [26]:
import numpy as np
x_final=np.array(emb_doc)
y_final=np.array(y)

In [27]:
x_final.shape,y_final.shape

((18285, 20), (18285,))

In [28]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_final,y_final,test_size=0.33,random_state=42)

In [29]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=20,batch_size=64)

Epoch 1/20
192/192 [==============================] - 7s 36ms/step - loss: 0.3196 - accuracy: 0.8461 - val_loss: 0.2047 - val_accuracy: 0.9132
Epoch 2/20
192/192 [==============================] - 6s 31ms/step - loss: 0.1276 - accuracy: 0.9528 - val_loss: 0.1968 - val_accuracy: 0.9190
Epoch 3/20
192/192 [==============================] - 6s 30ms/step - loss: 0.0777 - accuracy: 0.9729 - val_loss: 0.2323 - val_accuracy: 0.9167
Epoch 4/20
192/192 [==============================] - 6s 30ms/step - loss: 0.0468 - accuracy: 0.9858 - val_loss: 0.2996 - val_accuracy: 0.9171
Epoch 5/20
192/192 [==============================] - 6s 29ms/step - loss: 0.0277 - accuracy: 0.9911 - val_loss: 0.3222 - val_accuracy: 0.9135
Epoch 6/20
192/192 [==============================] - 6s 29ms/step - loss: 0.0189 - accuracy: 0.9949 - val_loss: 0.4202 - val_accuracy: 0.9102
Epoch 7/20
192/192 [==============================] - 6s 29ms/step - loss: 0.0125 - accuracy: 0.9965 - val_loss: 0.4818 - val_accuracy: 0.9060

In [30]:
y_pred=model.predict_classes(x_test)

PERFORMANCE MEASUREMENT

In [31]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[3127,  292],
       [ 257, 2359]])

In [32]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9090306545153273

In [33]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.92      0.91      0.92      3419
           1       0.89      0.90      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035



In [44]:
y_test

array([1, 0, 0, ..., 0, 1, 1])

In [45]:
y=np.resize(y_test,5200)


In [46]:
s=pd.read_csv('../input/fake-news/submit.csv')

In [47]:
submission=pd.DataFrame.from_dict({
    'id':s.id,
    'label':y
})

In [50]:
submission.to_csv("submit.csv",index=False)